In [3]:
import pandas as pd
import numpy as np
import re


# ===================== 1. 读取原始数据 =====================
def load_data(path="novel-1(1).csv"):
    """自动尝试常见中文编码"""
    encodings = ["utf-8-sig", "utf-8", "gb18030", "gbk"]
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc)
            print(f"✅ 成功使用编码 {enc} 读取文件，共 {len(df)} 行 {len(df.columns)} 列")
            return df
        except UnicodeDecodeError:
            print(f"⚠️ 编码 {enc} 读取失败，尝试下一个...")
    raise ValueError("❌ 无法读取文件，请确认 CSV 编码是否为 UTF-8 或 GB18030。")

# ===================== 2. 工具函数 =====================
def parse_date(date_str):
    try:
        if pd.isna(date_str) or date_str == "":
            return pd.NaT
        s = str(date_str).strip()
        m = re.search(r"\d{4}-\d{2}-\d{2}( \d{2}:\d{2}:\d{2})?", s)
        if m:
            s = m.group(0)
        return pd.to_datetime(s, errors="coerce")
    except Exception:
        return pd.NaT


def parse_int(val):
    try:
        if pd.isna(val):
            return 0
        s = str(val).replace(",", "").replace("，", "")
        s = re.sub(r"[^\d.-]", "", s)
        return int(float(s)) if s else 0
    except Exception:
        return 0


def parse_float(val):
    try:
        if pd.isna(val):
            return 0.0
        s = str(val).replace(",", "").replace("，", "")
        s = re.sub(r"[^\d.-]", "", s)
        return float(s) if s else 0.0
    except Exception:
        return 0.0


def parse_percentage(val):
    try:
        if pd.isna(val):
            return 0.0
        s = str(val).replace("%", "").replace("％", "")
        s = re.sub(r"[^\d.-]", "", s)
        return float(s) / 100 if s else 0.0
    except Exception:
        return 0.0






In [4]:
df = load_data()

✅ 成功使用编码 utf-8-sig 读取文件，共 83824 行 22 列


In [5]:
#检查缺失值
# 检查每一列是否有缺失值，如果有缺失值，删除掉缺失值所在行，并打印缺失值的数量
for col in df.columns:
    missing_count = df[col].isna().sum()
    if missing_count > 0:
        print(f"列 '{col}' 有 {missing_count} 个缺失值，正在删除对应行...")
        df = df[df[col].notna()].reset_index(drop=True)
print("缺失值处理完成。")

列 '作品视角' 有 373 个缺失值，正在删除对应行...
列 '所属系列' 有 317 个缺失值，正在删除对应行...
列 '当前被收藏数' 有 365 个缺失值，正在删除对应行...
缺失值处理完成。


In [6]:
if "发表时间" in df.columns:
    df["发表时间"] = df["发表时间"].apply(parse_date)
    print("✅ 发表时间处理完成")

# ---------- 类型 one-hot ----------
if "类型" in df.columns:
    all_types = set()
    for x in df["类型"].dropna():
        all_types.update([t.strip() for t in str(x).split("-") if t.strip()])
    print(f"共有 {len(all_types)} 种【类型】")

    type_dict = {}
    for t in all_types:
        safe = re.sub(r'[^\w]', '_', t)
        type_dict[f"类型_{safe}"] = df["类型"].apply(lambda x: 1 if pd.notna(x) and t in str(x) else 0)

    if type_dict:
        df = pd.concat([df, pd.DataFrame(type_dict, index=df.index)], axis=1)
    print("✅ 类型 one-hot 完成")

# ---------- 作品视角 one-hot ----------
if "作品视角" in df.columns:
    view_df = pd.get_dummies(df["作品视角"].fillna("未知"), prefix="视角")
    df = pd.concat([df, view_df], axis=1)
    print("✅ 作品视角 one-hot 完成")

# ---------- 内容标签 one-hot ----------
if "内容标签" in df.columns:
    # 统一分隔并收集所有标签
    tag_lists = df["内容标签"].fillna("").apply(lambda s: [t.strip() for t in str(s).replace('，', ',').split(',') if t.strip() and t.strip() != "无"])
    all_tags = sorted({tag for lst in tag_lists for tag in lst})
    print(f"共有 {len(all_tags)} 种标签")

    tag_dict = {}
    for tag in all_tags:
        safe = re.sub(r'[^\w]', '_', tag)
        tag_dict[f"标签_{safe}"] = tag_lists.apply(lambda lst: 1 if tag in lst else 0)

    if tag_dict:
        df = pd.concat([df, pd.DataFrame(tag_dict, index=df.index)], axis=1)
    print("✅ 内容标签 one-hot 完成")

# ---------- 版权转化 ----------
if "版权转化" in df.columns:
    # 标记是否有版权；同时批量生成具体版权one-hot
    banquan_lists = df["版权转化"].fillna("").apply(lambda s: [t.strip() for t in str(s).replace('，', ',').split(',') if t.strip() and t.strip() != "未签约"])
    all_banquan = sorted({bq for lst in banquan_lists for bq in lst})
    banquan_dict = {}
    for bq in all_banquan:
        safe = re.sub(r'[^\w]', '_', bq)
        banquan_dict[f"版权_{safe}"] = banquan_lists.apply(lambda lst: 1 if bq in lst else 0)

    if banquan_dict:
        df = pd.concat([df, pd.DataFrame(banquan_dict, index=df.index)], axis=1)

    df["有版权转化"] = df["版权转化"].apply(lambda x: 0 if pd.isna(x) or str(x).strip() == "未签约" or str(x).strip()=="" else 1)
    print("✅ 版权转化标记完成（1=有版权，0=未签约）")

# ---------- 签约状态 ----------
if "签约状态" in df.columns:
    df["签约状态_二分类"] = df["签约状态"].apply(
        lambda x: 0 if pd.notna(x) and str(x).strip() == "未签约" else 1
    )
    print("✅ 签约状态 二分类完成")

# ---------- 数值字段 ----------
num_cols = ["字数", "非v章节章均点击数", "总书评数", "当前被收藏数", "文章积分", "评价人数"]
for c in num_cols:
    if c in df.columns:
        df[c] = df[c].apply(parse_int)
print("✅ 数值列清洗完成")

# ---------- 评分 ----------
if "评分" in df.columns:
    df["评分"] = df["评分"].replace("暂无", np.nan).apply(parse_float)
    print("✅ 评分列处理完成")

# ---------- 百分比字段 ----------
pct_cols = ["五星比例", "四星比例", "三星比例", "二星比例", "一星比例"]
for c in pct_cols:
    if c in df.columns:
        df[c] = df[c].apply(parse_percentage)
print("✅ 百分比列处理完成")

# ---------- 输出结果 ----------
out_path = "novel_processed.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"\n🎯 数据处理完成，已保存到：{out_path}")
print(f"共 {len(df)} 条记录，{len(df.columns)} 个字段\n")
print("=== 数值列摘要 ===")
print(df.select_dtypes(include=["int64", "float64"]).describe())

✅ 发表时间处理完成
共有 39 种【类型】
✅ 类型 one-hot 完成
✅ 作品视角 one-hot 完成
共有 277 种标签
✅ 内容标签 one-hot 完成
✅ 版权转化标记完成（1=有版权，0=未签约）
✅ 签约状态 二分类完成
✅ 数值列清洗完成
✅ 评分列处理完成
✅ 百分比列处理完成

🎯 数据处理完成，已保存到：novel_processed.csv
共 82769 条记录，372 个字段

=== 数值列摘要 ===
                 字数     非v章节章均点击数          总书评数        当前被收藏数          文章积分  \
count  8.276900e+04  8.276900e+04  8.276900e+04  8.276900e+04  8.276900e+04   
mean   2.899567e+05  1.175291e+04  2.021193e+03  7.017412e+03  1.082868e+08   
std    2.568377e+05  3.673625e+04  1.699127e+04  2.367021e+04  4.501560e+08   
min    2.993400e+04  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    1.288610e+05  2.360000e+02  8.000000e+00  4.000000e+01  2.862823e+06   
50%    2.330670e+05  1.436000e+03  1.100000e+02  7.560000e+02  1.537090e+07   
75%    3.712420e+05  7.588000e+03  6.930000e+02  4.404000e+03  6.098224e+07   
max    6.622435e+06  1.693125e+06  2.246450e+06  1.734783e+06  4.030560e+10   

                 评分          评价人数          五星比例          四星比例   

In [7]:
# 删除作者、名称、发表时间数值完全相同的行（保留首次出现）
cols = ['作者', '名称', '发表时间']
missing = [c for c in cols if c not in df.columns]
if missing:
    print(f"⚠️ 无法去重，缺少列: {missing}")
else:
    # 规范化发表时间（转换为 datetime 并按秒四舍五入）以避免微小差异导致无法匹配
    df['__发表时间_norm'] = pd.to_datetime(df['发表时间'], errors='coerce').dt.round('s')
    before = len(df)
    df = df.drop_duplicates(subset=['作者', '名称', '__发表时间_norm'], keep='first').reset_index(drop=True)
    df = df.drop(columns='__发表时间_norm')
    after = len(df)
    print(f"去重完成：已删除 {before - after} 行，剩余 {after} 行")


去重完成：已删除 37 行，剩余 82732 行


In [8]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82732 entries, 0 to 82731
Columns: 372 entries, 作者 to 签约状态_二分类
dtypes: bool(10), datetime64[ns](1), float64(6), int64(346), object(9)
memory usage: 229.3+ MB


,字数,发表时间,非v章节章均点击数,总书评数,当前被收藏数,文章积分,评分,评价人数,五星比例,四星比例,...,版权_漫画签约,版权_电影签约,版权_电视剧签约,版权_短视频签约,版权_繁体出版_港_台_签约,版权_网络剧签约,版权_网络电影签约,版权_衍生品签约,有版权转化,签约状态_二分类
count,8.273200e+04,82732,8.273200e+04,8.273200e+04,8.273200e+04,8.273200e+04,82732.000000,82732.000000,82732.000000,82732.000000,...,82732.000000,82732.000000,82732.000000,82732.000000,82732.000000,82732.000000,82732.000000,82732.000000,82732.000000,82732.000000
mean,2.900055e+05,2023-06-04 12:49:01.377617152,1.175393e+04,2.021571e+03,7.017814e+03,1.082948e+08,5.918907,38.982558,0.357809,0.237103,...,0.000097,0.000060,0.000846,0.000012,0.003131,0.001257,0.000048,0.000314,0.022361,0.729246
min,2.993400e+04,2022-01-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.288735e+05,2022-08-22 00:00:00,2.360000e+02,8.000000e+00,4.000000e+01,2.863603e+06,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.331280e+05,2023-05-19 21:00:00.500000,1.436000e+03,1.100000e+02,7.570000e+02,1.537967e+07,8.200000,2.000000,0.310000,0.110000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,3.713295e+05,2024-03-10 13:32:27.500000,7.590000e+03,6.930000e+02,4.406000e+03,6.099243e+07,9.200000,17.000000,0.650000,0.450000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,6.622435e+06,2024-12-31 23:59:59,1.693125e+06,2.246450e+06,1.734783e+06,4.030560e+10,10.000000,46311.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,2.568768e+05,NaN,3.674034e+04,1.699491e+04,2.367228e+04,4.502209e+08,4.195260,258.916248,0.363078,0.282806,...,0.009833,0.007774,0.029076,0.003477,0.055864,0.035433,0.006953,0.017725,0.147857,0.444352
